In [20]:
from manim import *
import numpy as np

class CovariancePlaneProjection2D3D(ThreeDScene):
    def construct(self):
        # ---------------- 1️⃣ 3D Camera Setup ----------------
        self.set_camera_orientation(phi=75*DEGREES, theta=-60*DEGREES, distance=12)

        # ---------------- 2️⃣ Axes ----------------
        axes = ThreeDAxes(
            x_range=[-5,5,1],
            y_range=[-5,5,1],
            z_range=[-5,5,1],
            x_length=8, y_length=8, z_length=6
        )
        self.add(axes)

        # ---------------- 3️⃣ Curve and Points ----------------
        def curve_func(t):
            return np.array([np.cos(t)*2, np.sin(t)*2, np.sin(2*t)])

        t_vals = np.linspace(0, 2*np.pi, 12)
        points = np.array([curve_func(t) for t in t_vals])
        dots = VGroup(*[Dot3D(point=pt, color=BLUE, radius=0.12) for pt in points])
        self.play(FadeIn(dots))

        # ---------------- 4️⃣ Mean and HUD ----------------
        mean = np.mean(points, axis=0)
        mean_dot = Dot3D(point=mean, color=RED, radius=0.15)
        self.play(FadeIn(mean_dot))

        mean_label = MathTex(f"\\bar{{x}}=({mean[0]:.2f},{mean[1]:.2f},{mean[2]:.2f})").scale(0.6).to_corner(UL)
        cov_formula = MathTex(r"\Sigma = \frac{1}{N}\sum_i (x_i-\bar{x})(x_i-\bar{x})^T").scale(0.6).next_to(mean_label, DOWN, aligned_edge=LEFT)
        self.add_fixed_in_frame_mobjects(mean_label, cov_formula)

        # ---------------- 5️⃣ Covariance and Eigenvectors ----------------
        centered_points = points - mean
        cov = np.cov(centered_points.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        idx = np.argsort(eigvals)[::-1]
        eigvals = eigvals[idx]
        eigvecs = eigvecs[:, idx]

        # ---------------- 6️⃣ Planes, Variance, Projection, 2D View ----------------
        plane_size = 3
        colors = [YELLOW, GREEN]
        labels = ["High variance plane", "Low variance plane"]

        for i in range(2):
            # Choose eigenvectors for plane
            if i == 0:
                v1, v2 = eigvecs[:,0], eigvecs[:,1]  # high variance
            else:
                v1, v2 = eigvecs[:,1], eigvecs[:,2]  # low variance

            # Plane corners
            corners = [
                mean +  v1*plane_size +  v2*plane_size,
                mean +  v1*plane_size + -v2*plane_size,
                mean + -v1*plane_size + -v2*plane_size,
                mean + -v1*plane_size +  v2*plane_size,
            ]
            plane = Polygon(*corners, color=colors[i], fill_opacity=0.3)
            plane_label = MathTex(labels[i]).scale(0.6).to_corner(UR)
            self.add_fixed_in_frame_mobjects(plane_label)

            self.play(FadeIn(plane))
            self.wait(0.5)

            # Compute projected points
            projected_points = []
            proj_dots = VGroup()
            for pt in points:
                vec = pt - mean
                proj_vec = np.dot(vec, v1)*v1 + np.dot(vec, v2)*v2
                proj_pt = mean + proj_vec
                projected_points.append(proj_pt)
                proj_dots.add(Dot3D(point=proj_pt, color=ORANGE, radius=0.12))

            # Animate projection in 3D
            animations = [dots[j].animate.move_to(proj_dots[j].get_center()) for j in range(len(dots))]
            self.play(*animations, run_time=2)
            self.wait(0.5)

            # Compute variance for plane
            var_proj = np.sum([np.linalg.norm(pt - (mean + np.dot(pt-mean,v1)*v1 + np.dot(pt-mean,v2)*v2))**2 for pt in points])/len(points)
            var_text = MathTex(f"Variance = {var_proj:.2f}").scale(0.6).next_to(plane_label, DOWN, aligned_edge=RIGHT)
            self.add_fixed_in_frame_mobjects(var_text)
            self.wait(1)

            # ---------------- Switch to top-down 2D view ----------------
            self.move_camera(phi=0*DEGREES, theta=-90*DEGREES, distance=10)
            self.wait(1.5)

            # Show points clearly in 2D projection
            proj_2d_dots = VGroup(*[Dot3D(point=pt, color=ORANGE, radius=0.12) for pt in projected_points])
            self.play(FadeIn(proj_2d_dots))
            self.wait(1.5)
            self.play(FadeOut(proj_2d_dots))

            # ---------------- Back to 3D for next plane ----------------
            self.set_camera_orientation(phi=75*DEGREES, theta=-60*DEGREES, distance=12)
            self.play(FadeOut(plane), FadeOut(var_text), FadeOut(plane_label))
            self.wait(0.5)

        # ---------------- 7️⃣ Optional: Ambient 3D rotation ----------------
        self.begin_ambient_camera_rotation(rate=0.02)
        self.wait(5)
        self.stop_ambient_camera_rotation()


%manim -ql -v ERROR CovariancePlaneProjection2D3D


Manim Community v0.19.0